# LionAGI - Introduction 1 : Function Handler


LionAGI is an intelligent agent framework, it integrate big data with advanced Machine Learning models such as LLM. 

In this tutorial, we will go through **special function call handlers**

When handling big dataset, it is very common to use loops, but they can get complex quite quickly, and prone to errors. 
LionAGI provides a set of tools to handle various loops and other problems in function calling

Synchonous function call handlers:

- `to_list`  : Convert given object to list, additionally can flatten to 1-D and dropna
- `lcall`   : takes a list of inputs and apply a given function on each element 

Asynchonous function call handlers:

- `alcall` : Async List Call, similar to `lcall` but suitable for async
- `mcall`   : Mapped Call, map elements to functions, apply each function to corresponding element, or each element 
- `bcall`   : Batch Call, group elements into batches, apply function to each batch aysnchronously, and wait one batch to finish before next
- `tcall`   : Timed Call, delay, timeout, exception handling and calculates runtime
- `rcall`   : Retry Call, timed call plus retry with backoff and default value

In [1]:
import lionagi as li

### Synchrous Function Handlers

#### 1. to_list & lcall (list call)

In [2]:
# we use to_list, a helpful type converter
# it flattens a nested list, and also dropna

a = [
    1, None, 2, 3, 
    [4], 
    [5, None, 6]
]


li.to_list(a, flatten=True, dropna=True)

[1, 2, 3, 4, 5, 6]

In [3]:
# create a test input list
a = range(1,6)

# create some test functions
f1 = lambda x: x**2

# the first special function calling method is called l_call (list call)
# you can operate a single function on the whole set of input list
li.lcall(a, f1)

[1, 4, 9, 16, 25]

In [4]:
# you can also pass **kwargs of the functions
# the input for lcall should be a single list, (you can bundle various inputs in objects)

def multiply_and_increment(x, factor=1, increment=0):
    print(x * factor + increment)

li.lcall(a, multiply_and_increment, factor=2, increment=3);

5
7
9
11
13


### Async call handlers for Multi-Elements

LionAGI is designed to be async only, (sync functions also works)

there are many handlers for async operations

In [5]:
import asyncio

`alcall` (async list call)

In [6]:
# define an async function
async def add_1(x):
    return x+1

# let's try to run it as usual 
add_1(2)

<coroutine object add_1 at 0x1692a9d80>

In [7]:
# we need to add await in front of an async function to recieve the output, 
# rather than a coroutine object
async def async_inverse(x):
    return 1 / x if x != 0 else None

await async_inverse(2)

0.5

In [8]:
# similar to lcall, you can also pass in **kwargs
async def async_increment(x, inverse=False):
    if inverse:
        return await async_inverse(x+1)
    return x + 1

# we can use alcall (async list call) to run an async function on a list
# it uses to_list under the hood, so also can flatten and dropna the output

await li.alcall(
    input=[[1, 3], [7, 9]], 
    func=async_increment, 
    flatten=True, 
    inverse=True
)

[0.5, 0.25, 0.125, 0.1]

`mcall` (mapped call) allows you to map functions and arguments to a collection of objects, and call them all in parallel.


In [9]:
f0 = lambda x: x+2
f1 = lambda x: x*2
f2 = lambda x: x**2

# apply each function to its corresponding input
await li.mcall(input=[3,4,5], func=[f0, f1, f2])

[5, 8, 25]

In [10]:
# apply each function to every input hence 'explode'
await li.mcall(
    input=[3,4,5,6,7], 
    func=[f0, f1, f2], 
    explode=True
)

[[5, 6, 7, 8, 9], [6, 8, 10, 12, 14], [9, 16, 25, 36, 49]]

`bcall` batch call allows the previous batch to finish before the next gets sent

In [11]:
async def process_item(item):
    await asyncio.sleep(0.5)
    print(item*2)

inputs = [1, 2, 3, 4, 5]
batch_size = 2

# you should see the results get printed 2 items at a time, with a 0.5 second delay between each batch, 
# last batch should only have 1 element
await li.bcall(
    input=inputs, 
    func=process_item, 
    batch_size=2
);

2
4
6
8
10


In [12]:
async def process_item_with_exception(item):
    if item == 3:
        raise ValueError("Error processing item")
    await asyncio.sleep(0.5)
    print(item*2)
    return item * 2

# 5 and 6 didn't get processed because the function stopped after the second batch due to the exception
inputs = [1, 2, 3, 4, 5, 6]
batch_size = 2
try:
    results = await li.bcall(inputs, process_item_with_exception, batch_size)
    print(results)
except ValueError as e:
    print(f"Caught an exception: {e}")

2
4
Caught an exception: Error processing item


### Async Call Handlers for single element

`tcall` timed call Handle both synchronous and asynchronous calls with optional delay, error handling, and execution timing.

In [13]:
async def async_function(input):
    await asyncio.sleep(0.5)  # Simulate a delay
    if input == 'error':
        raise ValueError("Error triggered")
    return input.upper()

# you can add a delay at the tcall
await li.tcall(
    func=async_function, 
    input='hello',
    delay=0.1, 
    err_msg = 'Somehow Error', 
    ignore_err=False, 
    timing=True,                # also output how long it took for the function to run 
    timeout=None
)

8


('HELLO', 0.6022593975067139)

In [14]:
# you can add a delay at the tcall
await li.tcall(
    func=async_function, 
    input='error',
    err_msg = 'Somehow Failed', 
    ignore_err=True, 
)

Somehow Failed Error: Error triggered


`rcall` is a plus version of tcall, it includes

- retries
- delay
- backoff_factor
- default
- timeout

In [15]:
try:
    await li.rcall(
        func=async_function, 
        input='error',
        retries=3,
        backoff_factor=1,       # backoff factor 1, means we do not increase the wait time between retries
        delay=0.5               # the delay in rcall represents the time to wait after a failed attempt
    )
except Exception as e:
    print(f"Caught an exception: {e}")

An error occurred: Error triggered
An error occurred: Error triggered
An error occurred: Error triggered
Caught an exception: Error triggered
